# PacMan

## Observações de implementação 

- Mapas são registrados em um arquivo *.txt* e deve formatado corretamente com o mesmo número de caracteres em cada linha, apenas um pacman - posicao inicial (P) e pelo menos um objetivo (X)

- Estado definido por tupla (x, y) - posição no mapa

- Parametros para ajustes devem ser definidos na classe Maze

## Restrições para análise do projeto

- Mapas devem conter corredores de espessura de no máximo uma célula ?
- Não são permitidos movimentos na diagonal, apenas, cima, baixo, esquerda e direita
- Pacman não passa mais de uma vez pela mesma célula (busca deve levar em conta states explorados)

### Importações de bibliotecas

In [13]:
import time
from enum import Enum
from aima.search import *
from aima.utils import *

### Gerenciador do labirinto

In [14]:
class PosType(Enum):
    '''
    Enumerado com caracteres possiveis para representação do lobirinto
    '''
    FREE = ' '
    COIN = '.'
    B_COIN = 'o'
    WALL = '|'
    GHOST = 'G'
    PACMAN = 'P'
    GOAL = 'X'
    PATH = '+'

    @staticmethod
    def valid_inits():
        '''
        Retorna todos os enumerados validos que podem estar contidos em arquivo txt
        '''
        return [ptype for ptype in PosType if ptype not in [PosType.PATH]]

    @staticmethod
    def valid_pacman_positions():
        '''
        Retorna os enumerados onde o pacman pode se movimentar
        '''
        return [PosType.FREE, PosType.COIN, PosType.B_COIN, PosType.GOAL, PosType.PACMAN]

class Maze:
    '''
    Classe para ler arquivos txt representativos dos labirintos e comportamento dos elementos ao interagir
    com o pacman

    :self.w: largura do mapa
    :self.h: altura do mapa
    :self.initial_state: estado inicial (x, y)
    :self.goal_states: lista de estados objetivos [(x, y)]
    :self.free_cost: custo de uma célula livre
    :self.coin_cost: custo moeda simples
    :self.bcoin_cost: custo de uma moeda grande
    '''

    def __init__(self, filepath, free_cost=10, coin_cost=2, bcoin_cost=1):
        # Classe construtora para um maze - ler arquivo e guarda em uma lista de strings
        # Faz algumas checagens para prevenir formatos errados
        file = open(filepath, "r")
        self._maze = file.read().splitlines()
        file.close()
        self.w = len(self._maze[0])
        self.h = len(self._maze)
        self.initial_state = None
        self.goal_states = []
        self.free_cost = free_cost
        self.coin_cost = coin_cost
        self.bcoin_cost = bcoin_cost

        for y in range(self.h):
            if len(self._maze[y]) is not self.w:
                raise ValueError('Invalid map dimensions')
            else:
                # Percorre linhas obtidas pelo arquivo procurando por seugestao de posicao inicial do
                # pacman e os objetivos (se houverem), há uma simples checagem de formato invalido de mapa
                for x in range(self.w):
                    pos = (x, y)
                    if self.get(pos) is PosType.PACMAN:
                        if self.initial_state is None:
                            self.initial_state = pos
                        else:
                            raise ValueError('Multiple pacmans position')
                    elif self.get(pos) is PosType.GOAL:
                        self.goal_states.append(pos)
                    elif self.get(pos) not in PosType.valid_inits():
                        raise ValueError('Invalid char {}'.format(self._maze[y][x]))

                # Remove objetivos e pacman do mapa
                self._maze[y] = self._maze[y].replace(PosType.PACMAN.value, ' ').replace(PosType.GOAL.value, ' ')

    def get(self, pos):
        # definimos _maze[y][x]
        return PosType(self._maze[pos[1]][pos[0]])

    def get_cost(self, pos):
        # retorna o custo de acordo com a posicao
        ptype = self.get(pos)
        if ptype == PosType.COIN:
            return self.coin_cost
        elif ptype == PosType.B_COIN:
            return self.bcoin_cost
        else:
            return self.free_cost

    def possible_positions(self, pos):
        '''
        Retorna possiveis posicoes que o pacman consegue chegar a partir de pos
        :pos: posição do pacman (x, y)
        return lista de posicoes (x, y)
        '''
        x, y = pos[0], pos[1]
        possible_positions = []
        for step in [(0, -1), (1, 0), (0, 1), (-1, 0)]:  # up, right, down, left
            # soma os steps com x, e y, tomando cuidados com os limites transversais do mapa
            newX = self.w - 1 if x + step[0] < 0 else (x + step[0]) % self.w
            newY = self.h - 1 if y + step[1] < 0 else (y + step[1]) % self.h
            pos = (newX, newY)
            if self.get(pos) in PosType.valid_pacman_positions():
                possible_positions.append(pos)

        return possible_positions

    def __str__(self):
        string = ''
        for line in self._maze:
            string = string + line + '\n'
        return string

    @staticmethod
    def _print(maze):
        for line in maze:
            print(''.join(line))
        print('\n')

    def _copy_maze_list(self):
        maze = list()
        for line in self._maze:
            maze.append(list(line))
        return maze

    def print_solution(self, solution):
        '''
        Printa solucao
        :solution: lista de posicoes que levam a solucao [(x, y) ...]
        '''
        maze = self._copy_maze_list()
        for state in solution:
            maze[state[1]][state[0]] = PosType.PATH.value
        self._print(maze)

    def print_map(self):
        '''
        Printa mapa com posicao inical e objetivo
        '''
        maze = self._copy_maze_list()
        maze[self.initial_state[1]][self.initial_state[0]] = PosType.PACMAN.value
        for goal in self.goal_states:
            maze[goal[1]][goal[0]] = PosType.GOAL.value
        self._print(maze)

### Definição da classe problema

In [15]:
from aima.search import Problem

class MazePacmanProblem(Problem):
    '''
    Define o conjunto de ações, resultados, custo, e teste objetivo para o problema
    '''

    def __init__(self, maze):
        '''
        Instancia um objeto problema
        :initial: estado inicial do pacman - posicao por uma tupla (x, y)
        :goal: estado final, obejtivo do pacman - posição por uma tupla (x, y) ou lista de tuplas
        '''
        Problem.__init__(self, maze.initial_state, maze.goal_states)
        self.maze = maze

    def actions(self, state):
        return self.maze.possible_positions(state)

    def result(self, state, action):
        return action

    def goal_test(self, state):
        if isinstance(self.goal, list):
            return state in self.goal
        else:
            return state == self.goal

    def path_cost(self, cost_so_far, stateA, action, stateB):
        # custo do caminho é incrementado com o custo da proxima célula
        return cost_so_far + self.maze.get_cost(action)

### Classe auxiliar para medir eficiencias dos algoritmos

In [16]:
class Statistics:
    '''
    Classe para auxiliar/organizar as metragens de estatisticas dos algoritmos
    :self.iterations: numero de iteracoes
    :self.expanded: numero de nos expandidos pelo algoritmo
    :self.memory: pico de memoria (numero maximo de estados salvos ao mesmo tempo)
    :self.time: tempo em ms
    :self.path_cost: custo total da solução
    '''

    def __init__(self, iterations=0, expanded=1, memory=1):
        self.iterations = iterations
        self.expanded = expanded
        self.memory = memory
        self.time = None
        self.path_cost = None
        self._start_time = time.time()

    def update_iterations(self, increment):
        self.iterations = self.iterations + increment

    def update_expanded(self, increment):
        self.expanded = self.expanded + increment

    def update_memory(self, new_memory):
        if new_memory > self.memory:
            self.memory = new_memory

    def finish(self, path_cost):
        '''
        Finaliza o timer (aberto no construtor) e salva custo da solucao
        '''
        self.path_cost = path_cost
        self.time = time.time() - self._start_time

    def __str__(self):
        sb = []
        for key in self.__dict__:
            sb.append("{key}='{value}'".format(key=key, value=self.__dict__[key]))

        return ', '.join(sb)

    def __repr__(self):
        return self.__str__()

# BFS e DFS

In [17]:
def depth_first_search(problem):
    statistics = Statistics()

    frontier = [(Node(problem.initial))]

    explored = set()
    while frontier:
        statistics.update_memory(len(frontier))
        statistics.update_iterations(1)
        node = frontier.pop()
        if problem.goal_test(node.state):
            statistics.finish(node.path_cost)
            return statistics, node
        explored.add(node.state)
        childs = [child for child in node.expand(problem)
                  if child.state not in explored and child not in frontier]
        frontier.extend(childs)
        statistics.update_expanded(len(childs))

    statistics.finish_time(node.path_cost)
    return statistics, None


def breadth_first_search(problem):
    statistics = Statistics()
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        return node
    frontier = deque([node])
    explored = set()
    while frontier:
        statistics.update_memory(len(frontier))
        statistics.update_iterations(1)
        node = frontier.popleft()
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                if problem.goal_test(child.state):
                    statistics.finish(node.path_cost)
                    return statistics, child
                frontier.append(child)
                statistics.update_expanded(1)
    statistics.finish_time()
    return statistics, None

In [18]:
maze = Maze("mazes/maze.txt")
problem = MazePacmanProblem(maze)
print(maze)
maze.print_map()
statistics, node = breadth_first_search(problem)
print(statistics)
maze.print_solution(node.solution())

||||||||||||||||||||||||||||
.....||..............||.....
||||.||.||||||||||||.||.||||
||||.||.||||||||||||.||.||||
||||....||........||....||||
||||.||.||.||||||.||.||.||||
||||.||....||||||....||.||||
.....||.||.||||||.||.||.....
|.||.||.||........||.||.||.|
|.||.||.||||||||||||.||.||.|
|.||.||.||||||||||||.||.||.|
|oooooooooooooooooooooooooo|
|.|||||||.||||||||.|||||||o|
|.|||||||.||||||||.|||||||o|
|.||......||||||||......||o|
|.||.||||.||||||||.||||.||o|
|....||||.||||||||.||||oooo|
|||||||..............|||||||
|||||||.||||||||||||.|||||||
|.......|||||||||..........|
|.|||.||||........||||.|||.|
|.|||.||||.||||||.||||.|||.|
|.......||.||||||.||||.....|
|||||||.||.      ....|||||||
|||||||.|| ||||||.||.|||||||
|....||.||.||||||.||.||....|
|.||.||.||........||.||.||.|
|.||....||||||||||||....||.|
|.||.||.||||||||||||.||.||.|
|....||..............||... |
||||||||||||||||||||||||||||

||||||||||||||||||||||||||||
.....||..............||.....
||||.||.||||||||||||.||.||||
||||.||.|||||

In [19]:
maze = Maze("mazes/maze.txt")
problem = MazePacmanProblem(maze)
print(maze)
maze.print_map()
statistics, node = depth_first_search(problem)
print(statistics)
maze.print_solution(node.solution())

||||||||||||||||||||||||||||
.....||..............||.....
||||.||.||||||||||||.||.||||
||||.||.||||||||||||.||.||||
||||....||........||....||||
||||.||.||.||||||.||.||.||||
||||.||....||||||....||.||||
.....||.||.||||||.||.||.....
|.||.||.||........||.||.||.|
|.||.||.||||||||||||.||.||.|
|.||.||.||||||||||||.||.||.|
|oooooooooooooooooooooooooo|
|.|||||||.||||||||.|||||||o|
|.|||||||.||||||||.|||||||o|
|.||......||||||||......||o|
|.||.||||.||||||||.||||.||o|
|....||||.||||||||.||||oooo|
|||||||..............|||||||
|||||||.||||||||||||.|||||||
|.......|||||||||..........|
|.|||.||||........||||.|||.|
|.|||.||||.||||||.||||.|||.|
|.......||.||||||.||||.....|
|||||||.||.      ....|||||||
|||||||.|| ||||||.||.|||||||
|....||.||.||||||.||.||....|
|.||.||.||........||.||.||.|
|.||....||||||||||||....||.|
|.||.||.||||||||||||.||.||.|
|....||..............||... |
||||||||||||||||||||||||||||

||||||||||||||||||||||||||||
.....||..............||.....
||||.||.||||||||||||.||.||||
||||.||.|||||

### Best first search e Uniform cost search

In [20]:
### TODO: Use STATISTICS


def best_first_search(problem, f, display=False):
    f = memoize(f, 'f')
    node = Node(problem.initial)
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            if display:
                print(len(explored), "paths have been expanded and", len(frontier), "paths remain in the frontier")
            return node
        explored.add(node.state)
        for child in node.expand(problem):
            if child.state not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
                if f(child) < frontier[child]:
                    del frontier[child]
                    frontier.append(child)
    return None


def uniform_cost_search(problem, display=False):
    return best_first_search(problem, lambda node: node.path_cost, display)

In [21]:
maze = Maze("mazes/maze.txt")
problem = MazePacmanProblem(maze)
node = uniform_cost_search(problem)
maze.print_map()
maze.print_solution(node.solution())
print(len(node.solution()))

||||||||||||||||||||||||||||
.....||..............||.....
||||.||.||||||||||||.||.||||
||||.||.||||||||||||.||.||||
||||....||........||....||||
||||.||.||.||||||.||.||.||||
||||.||....||||||....||.||||
.....||.||.||||||.||.||.....
|.||.||.||........||.||.||.|
|.||.||.||||||||||||.||.||.|
|.||.||.||||||||||||.||.||.|
|oooooooooooooooooooooooooo|
|.|||||||.||||||||.|||||||o|
|.|||||||.||||||||.|||||||o|
|.||......||||||||......||o|
|.||.||||.||||||||.||||.||o|
|....||||.||||||||.||||oooo|
|||||||..............|||||||
|||||||.||||||||||||.|||||||
|.......|||||||||..........|
|.|||.||||........||||.|||.|
|.|||.||||.||||||.||||.|||.|
|.......||.||||||.||||.....|
|||||||.||.      ....|||||||
|||||||.||X||||||.||.|||||||
|....||.||.||||||.||.||....|
|.||.||.||........||.||.||.|
|.||....||||||||||||....||.|
|.||.||.||||||||||||.||.||.|
|....||..............||...P|
||||||||||||||||||||||||||||


||||||||||||||||||||||||||||
.....||..............||.....
||||.||.||||||||||||.||.||||
||||.||.||||

In [22]:
def hill_climbing(problem):
    statistics = Statistics()
    current = Node(problem.initial)
    while True:
        statistics.update_iterations(1)
        neighbors = current.expand(problem)
        if not neighbors:
            break
        neighbor = argmin_random_tie(neighbors, key=lambda node: problem.value(node.state))
        if problem.value(neighbor.state) > problem.value(current.state):
            break
        current = neighbor
        statistics.update_expanded(1)
    statistics.finish(1)
    if problem.goal_test(current.state):
        return statistics, current
    return statistics, None

### Hill climbing

In [23]:
maze = Maze("mazes/maze_himclimbing.txt")
problem = MazePacmanProblem(maze)
print(maze)
statistics, node = hill_climbing(problem)
print(statistics)
print(node)

maze = Maze("mazes/maze_himclimbing_2.txt")
problem = MazePacmanProblem(maze)
print(maze)
statistics, node = hill_climbing(problem)
print(statistics)
print(node)

|||||||||||
|. .......|
|.........|
|.o.......|
|..oo.....|
|....oo...|
|......oo.|
|........ |
|||||||||||



NotImplementedError: 